# 8.5 감성 분석(Sentiment Analysis)
## 8.5.1 감성 분석 소개
- 문서의 주관적인 감성, 의견, 감정, 기분 등을 파악하기 위한 방법
- 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용
- 문서 내 텍스트가 나타내는 여러가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산
    - 감성 지수: 긍정 감성 지수 + 부정 감성 지수 = 긍정 감성 or 부정 감성 결정
- 방법
    - 지도 학습: 텍스트 기반 이진 분류와 동일
    - 비지도 학습: 감성어휘사전(Lexiton) 이용해 문서의 긍정적, 부정적 감성 여부 판단

<B>*감성 어휘 사전 (Lexiton)</B>
- 감성 분석을 위한 용어와 문맥에 대한 다양한 정보 가지고 있음

## 8.5.2 지도학습 기반 감성 분석 실습 – IMDB 영화평
- 데이터: IMDB 영화 사이트 영화평 (www.kaggle.com/c/word2vec-nlp-tutorial/data)
- 모델: 영화평 텍스트 분석하여 감정(긍정 or 부정)예측 모델

In [5]:
!ls

8.5 감성 분석.ipynb
8.6 토픽 모델링(Topic Modeling) - 20 뉴스그룹.ipynb
8.7 문서 군집화 소개와 실습(Opinion Review 데이터 세트).ipynb
labeledTrainData.tsv
sampleSubmission.csv
testData.tsv
unlabeledTrainData.tsv
word2vec-nlp-tutorial


In [18]:
import pandas as pd

"""
* labeledTrainData.tsv // 학습 데이터
id: 각 데이터의 id
sentiment: 영화평(review)의 sentiment결과값(target label) -> 1: 긍정 평가, 0: 부정 평가
review: 영화평 텍스트
"""
review_df = pd.read_csv('./labeledTrainData.tsv', header=0, sep="\t", quoting=3) #tab으로 분리
review_df.head(3)


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [19]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [53]:
import re 
"""
*텍스트에서 피처로 별 의미 없는 것들 삭제
- HTML 형식에서 추출 하여 <br/> 존재 -> 삭제 
- 숫자, 특수문자 -> 공백으로 변경
"""

# <br> html 태그는 replace 함수로 공백으로 변환 // Dataframe/Series 문자열 연산을 위한 str 속성 제공, replace() 적용
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환 // lambda식으로 re.sub() 적용
# re.sub(pattern, repl, string): string에서 pattern과 매치하는 텍스트를 repl로 치환
# [^a-zA-Z]: 영어 대소문자가 아닌 모든 문자를 찾음
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )

In [54]:
print(review_df['review'][0])

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

In [24]:
from sklearn.model_selection import train_test_split

# 결정값 클래스(sentiment) 컬럼을 추출해 결정값 데이터 세트 만듦
class_df = review_df['sentiment']
# 원본 데이터 세트에서 id와 sentiment 컬럼을 삭제해 피처 데이터 세트 생성
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

#학습용, 테스트용 데이터 세트로 분리
X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape # ((17500, 1), (7500, 1)) --> 학습용: 175,000개 리뷰, 테스트용: 7,500개 리뷰

((17500, 1), (7500, 1))

### 감성평 텍스트 피쳐 벡터화 -> ML 분류 알고리즘 적용 -> 예측 성능 측정 
1. Pipeline 객체 이용
    - Count 벡터화 적용 성능 확인
    - TD-IDF 벡터화 적용 성능 확인
2. Classifier
    - LogisticRegression
3. 예측 성능 평가
    - 이진분류 -> 정확도, ROC-AUC 모두 측정
4. 비교 결과
    - TD-IDF 기반 피쳐 벡터화가 예측 성능이 조금 더 나음

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

"""
* CountVectorizer
stop_words='english' // 영어 스톱 워드로 지정된 단어는 추출에서 제외
ngram_range=(1,2) // BOW 단어 순서 보강 위한 피쳐 추출 방법(범위 최솟값, 범위 최댓값), (1, 2):토큰화된 단어를 1개씩, 순서대로 2개씩 묶어서 추출
""" 
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. //정규화 강도를 결정하는 parameter. C값이 낮을수록 계수를 0으로 근사하므로 정규화 강화
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])


# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

/Users/csg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


예측 정확도는 0.8873, ROC-AUC는 0.9503


In [14]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

/Users/csg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


예측 정확도는 0.8936, ROC-AUC는 0.9598


## 8.5.3 비지도학습 기반 감성 분석 소개
- 감성 분석용 데이터는 실제 결정된 레이블 값을 가지고 있지 않음 -> 지도 학습 적용 어려움
- 비지도 감성 분석: Lexiton 기반 -> 한글용 Lexiton이 없음

<b>* Lexiton</b>
- 어휘집 -> 감성 어휘 사전(감성 사전)
- 감성 지수(Polarity score)
    - 긍정(Positive) 감성 or 부정(Negative) 감성의 정도를 의미하는 수치
    - 단어의 위치, 주변 단어, 문맥, POS(Part of Speech)등을 참고해 결정
- NLTK 패키지에서 지원 // 파이썬 자연어 패키지

<b>* WordNet</b>
- NLTK 패키지 내 WordNet 모듈
- 영어 어휘 사전
- 시맨틱(semantic, 문맥상 의미) 분석 제공
   - 예) present: 선물, 현재 
   - Sysnet(Sets of cognitive synonyms): 각각 품사(명사, 동사, 형용사, 부사 등)로 구성된 개별 단어의 문맥, 시맨틱 정보를 제공 

<b>* 대표적인 감성 사전</b>
- NLTK 감성 사전의 예측 성능이 좋지 않음

<b>1. SentiWordNet</b>
- 감성 단어 전용의 WordNet
- WordNet의 Sysnet별로 3가지 감성 점수(sentiment socre) 할당
    - 긍정 감성 지수: 얼마나 긍정적인가?
    - 부정 감성 지수: 얼마나 부정적인가?
    - 객관성 지수: 감성과 관계없이 얼마나 객관적인가?
- 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수를 계산하고, 이를 기반으로 감성이 긍정인지 부정인지 결정

<b> 2. VADER </b>
- 소셜미디어 텍스트에 대한 감성 분석 제공 패키지
- 뛰어난 감성 분석 결과, 빠른 수행시간, 대용량 텍스트 데이터에 잘 사용

<b> 3. Pattern </b>
- 예측 성능 좋음 
- 파이썬 3.x 호환 안됨

<b> SentiWordNet, VADER 감성 사전 감성 분석 예측 성능 분석 -> 지도 학습 기반 분류와 성능 비교 </b>
- WordNet: 시맨틱 기반 사전 구축 방법 설명
- SentiWorkdNet: 감성 사전 구성 방식 설명

## 8.5.4 SentiWordNet을 이용한 Sentiment Analysis 
* WordNet Synset 클래스의 이해
* SentiWordNet SentiSynset 클래스의 이해

In [58]:
import nltk
# NLTK 모든 데이터 세트와 패키지 받음 -> WordNet 서브 패키지 이용
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/csg/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /Users/csg/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /Users/csg/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]    |     /Users/csg/nltk_data

[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package state_union is already up-to-date!
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /Users/csg/nltk_data...
[nltk_data]    |   Package stopwo

True

In [28]:
from nltk.corpus import wordnet as wn

term = 'present'

"""
*sysnets()
- 서로 다른 semantic가지는 sysnet 객체 반환
- sysnet 객체
    - Sysnet(POS 태그)
    - POS 태그: 의미.품사.구분 인덱스(동일 품사내 의미가 여러가지 일 경우) (예) give.n.08
"""
# 'present'라는 단어로 wordnet의 synsets 생성. 
synsets = wn.synsets(term) # sysnets(): 파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Sysnet 객체를 리스트로 반환
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 갯수:', len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 갯수: 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [32]:
# sysnet 객체 속성을 이용해 시맨틱적 요소 표현
for synset in synsets :
    print('##### Synset name : ', synset.name(),'#####')
    print('POS :',synset.lexname()) # POS(Part of Speech): 품사
    print('Definition:',synset.definition()) # definition: 정의
    print('Lemmas:',synset.lemma_names()) # lemma: 부명제
    print('--------------------------------------------')

##### Synset name :  present.n.01 #####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
--------------------------------------------
##### Synset name :  present.n.02 #####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
--------------------------------------------
##### Synset name :  present.n.03 #####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
--------------------------------------------
##### Synset name :  show.v.01 #####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
--------------------------------------------
##### Synset name :  present.v.02 #####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['pr

In [48]:
# Wordnet 어휘 간 관계를 유사도로 출력 가능 -> path_similarity() 메서드 제공

# synset 객체를 단어별로 생성합니다. 
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree , lion , tiger , cat , dog]
similarities = []
entity_names = [ entity.name().split('.')[0] for entity in entities]

# 단어별 synset 들을 iteration 하면서 다른 단어들의 synset과 유사도를 측정합니다. 
for entity in entities:
    similarity = [ round(entity.path_similarity(compared_entity), 2)  for compared_entity in entities ]
    similarities.append(similarity)
    
# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame형태로 저장합니다.  
similarity_df = pd.DataFrame(similarities , columns=entity_names, index=entity_names)
similarity_df # lion은 tiger와 유사도가 가장 큼(0.33), tree와 가장 작음(0.07)

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [49]:
import nltk
from nltk.corpus import sentiwordnet as swn # wordnet의 synset과 유사

"""
* senti_synsets()
- senti_synset 객체 반환
    - SentiSysnet(POS 태그)
    - POS 태그: 의미.품사.구분 인덱스(동일 품사내 의미가 여러가지 일 경우) (예) give.n.08
"""
senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [50]:
import nltk
from nltk.corpus import sentiwordnet as swn

# SentiSysnet 객체 속성을 이용해 감성 지수(긍정 지수, 부정 지수), 객관성 지수 표현 -> 객관성 지수:1 이면 감성 지수:0
father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score()) # 긍정 감성 지수
print('father 부정감성 지수: ', father.neg_score()) # 부정 감성 지수
print('father 객관성 지수: ', father.obj_score()) # 객관성 지수
print('\n')
fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous .pos_score()) # 긍정 감성 지수
print('fabulous 부정감성 지수: ',fabulous .neg_score()) # 부정 감성 지수
print('fabulous 객관성 지수: ', fabulous.obj_score()) # 객관성 지수

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
fabulous 객관성 지수:  0.0


### SentiWordNet Lexicon을 이용한 영화 감상평 감성 분석
- 데이터: IMDB 영화 감상평 감성 분석
- 순서
    1. 문서를 문장 단위로 분해
    2. 문장을 단어 단위로 토큰화 하고 품사 태깅
    3. 품사 태깅된 단어 기반으로 sysnet 객체와 senti_synset 객체 생성
    4. Senti_synset에서 긍정 감성, 부정 감성 지수를 구하고 합한 뒤, 특정 임계치값 이상일 경우 긍정 or 부정으로 결정

In [62]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환 함수
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ # 형용사
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV # 부사
    elif tag.startswith('V'):
        return wn.VERB
    return 

In [63]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

# 문서를 문장으로 -> 단어 토큰 -> 품사 태깅 -> senti_synset 클래스 생성 -> polarity score 합산 함수
def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    
    raw_sentences = sent_tokenize(text)
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산 
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출  
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word , tag in tagged_sentence:
            
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN , wn.ADJ, wn.ADV):
                continue 
                
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성. 
            synsets = wn.synsets(lemma , pos=wn_tag)
            if not synsets:
                continue
                
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산. 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())           
            tokens_count += 1
    
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0 :
        return 1
    
    return 0

In [64]:
review_df['preds'] = review_df['review'].apply( lambda x : swn_polarity(x) )
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [65]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score

# 분류 성능 평가 함수
def get_clf_eval(y_test=None, pred=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [66]:
print('#### SentiWordNet 예측 성능 평가 ####')
get_clf_eval(y_target, preds)

#### SentiWordNet 예측 성능 평가 ####
오차 행렬
[[7668 4832]
 [3636 8864]]
정확도: 0.6613, 정밀도: 0.6472, 재현율: 0.7091,    F1: 0.6767, AUC:0.6613


## 8.5.5 VADER을 이용한 Sentiment Analysis
* VDAER Lexicon
    - 소셜 미디어 감성 분석 용도로 만들어진 룰기반 Lexicon
    - NLTK 패키지 내 SentimentIntensityAnalyzer 클래스 이용 or vaderSentiment 패키지 pip 설치

In [67]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

"""
* senti_analyzer.polarity_scores
- 딕셔너리 형태의 감성 지수 반환
- neg: 부정 지수, neu: 중립 지수, pos: 긍정 지수
- compound: neg, neu, pos조합해 -1 ~ 1사이 감성 지수 -> 보통 0.1이상 긍정, 그 이하 부정 or 임계값 조정해 예측 성능 맞춤
"""
senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0]) # 딕셔너리 형태의 감성 지수 반환
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


### VADER Lexicon을 이용한 영화 감상평 감성 분석
- 데이터: IMDB 영화 감상평 감성 분석

In [70]:
# 영화 감상평 텍스트, 임계값 전달 -> 감성 결과 반환
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

In [71]:
# apply lambda 식을 이용하여 레코드별로 vader_polarity( )를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

In [72]:
print('#### VADER 예측 성능 평가 ####')
get_clf_eval(y_target, vader_preds)

#### VADER 예측 성능 평가 ####
오차 행렬
[[ 6736  5764]
 [ 1867 10633]]
정확도: 0.6948, 정밀도: 0.6485, 재현율: 0.8506,    F1: 0.7359, AUC:0.6948
